In [1]:
import pandas as pd
import numpy as np

from sklearn.datasets import fetch_california_housing

from evidently import ColumnMapping

from evidently.report import Report
from evidently.metrics.base_metric import generate_column_metrics
from evidently.metric_preset import DataDriftPreset, TargetDriftPreset, DataQualityPreset, RegressionPreset
from evidently.metrics import *

from evidently.test_suite import TestSuite
from evidently.tests.base_test import generate_column_tests
from evidently.test_preset import DataStabilityTestPreset, NoTargetPerformanceTestPreset, RegressionTestPreset
from evidently.tests import *

In [12]:
# Charger les données
data_train = pd.read_csv("app_train.csv")
data_test = pd.read_csv("app_test.csv")

In [16]:
data_train.columns

Index(['Unnamed: 0', 'SK_ID_CURR', 'TARGET', 'NAME_CONTRACT_TYPE',
       'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'CNT_CHILDREN',
       'AMT_INCOME_TOTAL', 'AMT_CREDIT',
       ...
       'FLAG_DOCUMENT_18', 'FLAG_DOCUMENT_19', 'FLAG_DOCUMENT_20',
       'FLAG_DOCUMENT_21', 'AMT_REQ_CREDIT_BUREAU_HOUR',
       'AMT_REQ_CREDIT_BUREAU_DAY', 'AMT_REQ_CREDIT_BUREAU_WEEK',
       'AMT_REQ_CREDIT_BUREAU_MON', 'AMT_REQ_CREDIT_BUREAU_QRT',
       'AMT_REQ_CREDIT_BUREAU_YEAR'],
      dtype='object', length=123)

In [14]:
# import warnings
# warnings.filterwarnings('ignore')
# warnings.simplefilter('ignore')
# # Load Data
# data = fetch_california_housing(as_frame=True)
# housing_data = data.frame
# housing_data.rename(columns={'MedHouseVal': 'target'}, inplace=True)
# housing_data['prediction'] = housing_data['target'].values + np.random.normal(0, 5, housing_data.shape[0])
# reference = housing_data.sample(n=5000, replace=False)
# current = housing_data.sample(n=5000, replace=False)



# Assurez-vous d'avoir correctement chargé votre DataFrame client auparavant
# Votre DataFrame client est supposé être stocké dans une variable appelée 'client_data'

# Supprimez ces lignes si elles ne sont pas nécessaires
# Elles désactivent simplement les avertissements de pandas
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

# Utilisez votre DataFrame client (client_data) à la place de la charge de données California Housing
client_data = data_train.copy()  # Assurez-vous d'avoir une copie de vos données

# Assurez-vous que les colonnes correspondent à celles de votre DataFrame client
client_data.rename(columns={'TARGET': 'target'}, inplace=True)

# Ajoutez une colonne de prédiction basée sur la colonne 'target' avec un bruit aléatoire
client_data['prediction'] = client_data['target'] + np.random.normal(0, 5, client_data.shape[0])

# Créez un DataFrame de référence et actuel à partir de votre DataFrame client
reference = client_data.sample(n=5000, replace=False)
current = client_data.sample(n=5000, replace=False)

# Affichez les DataFrames de référence et actuels
print("DataFrame de Référence (Reference):")
print(reference)
print("\nDataFrame Actuel (Current):")
print(current)


DataFrame de Référence (Reference):
      Unnamed: 0  SK_ID_CURR  target NAME_CONTRACT_TYPE CODE_GENDER  \
5338        5338      106241       1         Cash loans           M   
1989        1989      102331       0    Revolving loans           F   
383          383      100441       0         Cash loans           F   
155          155      100182       0    Revolving loans           F   
6324        6324      107379       0         Cash loans           F   
...          ...         ...     ...                ...         ...   
143          143      100165       0         Cash loans           F   
3412        3412      103986       1         Cash loans           F   
5201        5201      106085       0         Cash loans           F   
4941        4941      105782       0         Cash loans           M   
3095        3095      103616       0         Cash loans           F   

     FLAG_OWN_CAR FLAG_OWN_REALTY  CNT_CHILDREN  AMT_INCOME_TOTAL  AMT_CREDIT  \
5338            Y             

In [10]:
housing_data.head(10)

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,target,prediction
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526,1.148991
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585,2.405002
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521,11.201690
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413,6.620301
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422,0.419609
5,4.0368,52.0,4.761658,1.103627,413.0,2.139896,37.85,-122.25,2.697,15.176102
6,3.6591,52.0,4.931907,0.951362,1094.0,2.128405,37.84,-122.25,2.992,8.170849
7,3.1200,52.0,4.797527,1.061824,1157.0,1.788253,37.84,-122.25,2.414,6.040389
8,2.0804,42.0,4.294118,1.117647,1206.0,2.026891,37.84,-122.26,2.267,-3.229664
9,3.6912,52.0,4.970588,0.990196,1551.0,2.172269,37.84,-122.25,2.611,-0.145164


In [ ]:
# Importation des bibliothèques nécessaires
# from datadrift import Report, DataDriftPreset, ColumnSummaryMetric, ColumnQuantileMetric, ColumnDriftMetric, generate_column_metrics

# Définition du rapport initial avec des métriques de référence
report = Report(metrics=[
    DataDriftPreset(),  # Utilise la métrique de dérive de données prédéfinie
])

# Exécution du rapport avec des données de référence et de données actuelles
report.run(reference_data=reference, current_data=current)

# Affichage du rapport
report

# Création d'un rapport avec des métriques spécifiques sur la colonne 'AveRooms'
report = Report(metrics=[
    ColumnSummaryMetric(column_name='AMT_INCOME_TOTAL'),  # Résume les statistiques de la colonne
    ColumnQuantileMetric(column_name='AMT_INCOME_TOTAL', quantile=0.25),  # Calcule le quantile de la colonne
    ColumnDriftMetric(column_name='AMT_INCOME_TOTAL'),  # Calcule la dérive de la colonne
])

# Exécution du rapport avec des données de référence et de données actuelles
report.run(reference_data=reference, current_data=current)

# Affichage du rapport
report

# Création d'un rapport avec des métriques générées pour plusieurs colonnes
report = Report(metrics=[
    generate_column_metrics(ColumnQuantileMetric, parameters={'quantile': 0.25}, columns=['AMT_INCOME_TOTAL', 'AMT_CREDIT']),
    # Génère des métriques de quantile pour les colonnes spécifiées
])

# Exécution du rapport avec des données de référence et de données actuelles
report.run(reference_data=reference, current_data=current)

# Affichage du rapport
report

# Création d'un rapport avec une variété de métriques pour différentes colonnes et une métrique prédéfinie
report = Report(metrics=[
    ColumnSummaryMetric(column_name='AMT_INCOME_TOTAL'),  # Résume les statistiques de la colonne
    generate_column_metrics(ColumnQuantileMetric, parameters={'quantile': 0.25}, columns='num'),  # Génère des métriques de quantile pour 'num'
    DataDriftPreset(),  # Utilise la métrique de dérive de données prédéfinie
])

# Exécution du rapport avec des données de référence et de données actuelles
report.run(reference_data=reference, current_data=current)

# Affichage du rapport
report

# Conversion du rapport en dictionnaire
report.as_dict()

# Conversion du rapport en format JSON
report.json()

# Explication du code :
# Ce code Python utilise la bibliothèque "datadrift" pour créer des rapports de dérive de données entre un jeu de données de référence (reference) et un jeu de données actuel (current).
# Le code commence par importer les bibliothèques nécessaires.
# Il définit ensuite plusieurs rapports avec différentes métriques pour évaluer la dérive de données, telles que la dérive globale des données, les statistiques de colonnes, les quantiles, etc.
# Chaque rapport est exécuté avec les données de référence et les données actuelles, puis le rapport est affiché.
# À la fin, le code montre comment convertir le rapport en dictionnaire et en format JSON.
# Ces rapports sont utiles pour surveiller et analyser la stabilité des données au fil du temps, ce qui est essentiel dans de nombreuses applications, notamment pour détecter des anomalies ou des changements dans les données.

# # Supposons que reference et current contiennent vos données de référence et actuelles

# # Analyse de la dérive de données
# data_drift_report = e.Report(
#     metrics=[e.DataDriftPreset()]
# )
# data_drift_report.run(reference_data=reference, current_data=current)

# # Analyse de la colonne 'AMT_INCOME_TOTAL'
# income_total_report = e.Report(
#     metrics=[
#         e.ColumnSummaryMetric(column_name='AMT_INCOME_TOTAL'),
#         e.ColumnQuantileMetric(column_name='AMT_INCOME_TOTAL', quantile=0.25),
#         e.ColumnDriftMetric(column_name='AMT_INCOME_TOTAL')
#     ]
# )
# income_total_report.run(reference_data=reference, current_data=current)

# # Analyse des colonnes 'AMT_INCOME_TOTAL' et 'AMT_CREDIT'
# income_credit_report = e.Report(
#     metrics=[
#         e.generate_column_metrics(e.ColumnQuantileMetric, parameters={'quantile': 0.25}, columns=['AMT_INCOME_TOTAL', 'AMT_CREDIT'])
#     ]
# )
# income_credit_report.run(reference_data=reference, current_data=current)

# # Analyse de la dérive de données avec la colonne 'AMT_INCOME_TOTAL'
# income_data_drift_report = e.Report(
#     metrics=[
#         e.ColumnSummaryMetric(column_name='AMT_INCOME_TOTAL'),
#         e.generate_column_metrics(e.ColumnQuantileMetric, parameters={'quantile': 0.25}, columns='AMT_INCOME_TOTAL'),
#         e.DataDriftPreset()
#     ]
# )
# income_data_drift_report.run(reference_data=reference, current_data=current)

# Affichage des rapports
income_total_report.show()
income_credit_report.show()
income_data_drift_report.show()


In [5]:
# Test Suite
tests = TestSuite(tests=[
    TestNumberOfColumnsWithMissingValues(),
    TestNumberOfRowsWithMissingValues(),
    TestNumberOfConstantColumns(),
    TestNumberOfDuplicatedRows(),
    TestNumberOfDuplicatedColumns(),
    TestColumnsType(),
    TestNumberOfDriftedColumns(),
])

tests.run(reference_data=reference, current_data=current)
tests
suite = TestSuite(tests=[
    NoTargetPerformanceTestPreset(),
])

suite.run(reference_data=reference, current_data=current)
suite
suite = TestSuite(tests=[
    TestColumnDrift('Population'),
    TestMeanInNSigmas('HouseAge'),
    NoTargetPerformanceTestPreset(columns=['AveRooms', 'AveBedrms', 'AveOccup'])
])

suite.run(reference_data=reference, current_data=current)
suite
suite = TestSuite(tests=[
    TestNumberOfColumnsWithMissingValues(),
    TestNumberOfRowsWithMissingValues(),
    TestNumberOfConstantColumns(),
    TestNumberOfDuplicatedRows(),
    TestNumberOfDuplicatedColumns(),
    TestColumnsType(),
    TestNumberOfDriftedColumns(),
    TestColumnDrift('Population'),
    TestShareOfOutRangeValues('Population'),
    DataStabilityTestPreset(),
    RegressionTestPreset()
    
])

suite.run(reference_data=reference, current_data=current)
suite
suite.as_dict()
suite.json()
#suite.save_html('test_suite.html')

'{"version": "0.4.8", "tests": [{"name": "The Number of Columns With Missing Values", "description": "The number of columns with missing values is 0. The test threshold is lte=0.", "status": "SUCCESS", "group": "data_integrity", "parameters": {"condition": {"lte": 0}, "value": 0.0}}, {"name": "The Number Of Rows With Missing Values", "description": "The number of rows with missing values is 0. The test threshold is lte=0 \\u00b1 1e-12.", "status": "SUCCESS", "group": "data_integrity", "parameters": {"condition": {"lte": {"value": 0.0, "relative": 0.1, "absolute": 1e-12}}, "value": 0.0}}, {"name": "Number of Constant Columns", "description": "The number of constant columns is 0. The test threshold is lte=0.", "status": "SUCCESS", "group": "data_integrity", "parameters": {"condition": {"lte": 0}, "value": 0.0}}, {"name": "Number of Duplicate Rows", "description": "The number of duplicate rows is 0. The test threshold is eq=0 \\u00b1 1e-12.", "status": "SUCCESS", "group": "data_integrity"

In [6]:
suite.save_html('test_suite.html')